In [2]:
import numpy as np
import pandas as pd
from bert_serving.client import BertClient
from sklearn.feature_extraction.text import TfidfVectorizer # 创建TF-IDF向量化器
from cnsenti import Sentiment

In [6]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [61]:
corpus_18_20_path ='F:\\zzqaq\\data\\corpus_2018_2020.csv'
corpus_1820 = pd.read_csv(corpus_18_20_path,encoding="gb18030",converters={'code':str,'year':str,'board':str})
corpus_1820[:2]

,code,year,corpus,board,sentence_count
0,000004,2018,讨论 分析 概述 年度 实现 主营业务 收入 万元 上年 同期 增加 实现 归属于 上市公司...,00,52
1,000004,2020,讨论 分析 概述 年 业务 转型 后 第一年 年 筹划 发行 股份 收购 智游 网安 股权 ...,00,126


### 提取前512关键词

基于统计的关键词提取方法中最常用的词频.逆文档频率（TF-IDF）算法对关键词进行提取，TF-IDF算法可以评估某个词语对于一个语料库中的某一段文本的重要程度。

TF-IDF权重的大小不仅取决于一个词语在当前文档中出现的频率，还取决于它在整个语料库中的出现频率。如果一个词语在整个语料库中出现的频率较高，则认为它的区分度较低，应该被赋予较低的权重。


In [58]:
# 提取重要性前512词语
corpus = corpus_1820['corpus']
vectorizer = TfidfVectorizer(max_features=512)
X = vectorizer.fit_transform(corpus) 

features_string = []
for i in range(X.shape[0]):
    # 获取当前文本数据的TF-IDF值
    tfidf = X[i,:].toarray().squeeze()
    # 获取TF-IDF值从高到低排名的索引
    sorted_indices = tfidf.argsort()[::-1]  # argsort()方法并取反，以便获取从高到低排名的索引
    # 输出当前文本数据的前2个TF-IDF值高的特征单词
    top_indices = sorted_indices
    top_features = [vectorizer.get_feature_names()[i] for i in top_indices] # 特征单词
#     top_tfidf = [tfidf[i] for i in top_indices] # 特征向量
    string = ' '.join(top_features)
    features_string.append(string)
# 空格数
features_string[0].count(' ')

511

In [63]:
# 查看
top_features_512 = corpus_1820.copy()
top_features_512.corpus = features_string
top_features_512.head(2)

,code,year,corpus,board,sentence_count
0,000004,2018,适用 有限公司 股权 医药 业务 情况 科技股份 研发 资产 营业 出售 投资 子公司 行业...,00,52
1,000004,2020,安全 应用 检测 工业 平台 科技股份 适用 行业 有限公司 进行 互联网 情况 研发 数据...,00,126


In [64]:
# 保存
top_features_512.to_csv('F:\\zzqaq\\data\\top_features_512.csv',encoding="gb18030",index = False)

In [10]:
# 读取
top_features_512_path ='F:\\zzqaq\\data\\top_features_512.csv'
top_features_512 = pd.read_csv(top_features_512_path,encoding="gb18030",converters={'code':str,'year':str,'board':str})
top_features_512.head(2)

,code,year,corpus,board,sentence_count
0,000004,2018,适用 有限公司 股权 医药 业务 情况 科技股份 研发 资产 营业 出售 投资 子公司 行业...,00,52
1,000004,2020,安全 应用 检测 工业 平台 科技股份 适用 行业 有限公司 进行 互联网 情况 研发 数据...,00,126


In [11]:
len(top_features_512.corpus)

7781

—————————————————————————————  
### bert预训练生成词向量 🙂  
参考：https://cloud.tencent.com/developer/article/2094948#:~:text=BERT%20%E6%A8%A1%E5%9E%8B%E6%98%AF%E4%B8%80%E7%A7%8D%20NLP%20%E9%A2%84%E8%AE%AD%E7%BB%83%E6%8A%80%E6%9C%AF%EF%BC%8C%E6%9C%AC%E6%96%87%E4%B8%8D%E4%BB%8B%E7%BB%8D%20BERT%20%E7%9A%84%E5%8E%9F%E7%90%86%EF%BC%8C%E4%B8%BB%E8%A6%81%E5%85%B3%E6%B3%A8%E5%A6%82%E4%BD%95%E5%BF%AB%E9%80%9F%E4%B8%8A%E6%89%8B%E4%BD%BF%E7%94%A8%20BERT%20%E6%A8%A1%E5%9E%8B%E7%94%9F%E6%88%90%E8%AF%8D%E5%90%91%E9%87%8F%E7%94%A8%E4%BA%8E%E4%B8%8B%E6%B8%B8%E4%BB%BB%E5%8A%A1%E3%80%82,Google%20%E5%B7%B2%E7%BB%8F%E5%85%AC%E5%BC%80%E4%BA%86%20TensorFlow%20%E7%89%88%E6%9C%AC%E7%9A%84%20%E9%A2%84%E8%AE%AD%E7%BB%83%E6%A8%A1%E5%9E%8B%E5%92%8C%E4%BB%A3%E7%A0%81%20%EF%BC%8C%E5%8F%AF%E4%BB%A5%E7%94%A8%E4%BA%8E%E7%94%9F%E6%88%90%E8%AF%8D%E5%90%91%E9%87%8F%EF%BC%8C%E4%BD%86%E6%98%AF%E8%BF%98%E6%9C%89%E6%9B%B4%E7%AE%80%E5%8D%95%E7%9A%84%E6%96%B9%E6%B3%95%EF%BC%9A%E7%9B%B4%E6%8E%A5%E8%B0%83%E7%94%A8%E5%B0%81%E8%A3%85%E5%A5%BD%E7%9A%84%E5%BA%93%20bert-as-service%20%E3%80%82  
    
#### 低版本tf环境配置，tf版本与conda配套    
bert-serving-start 配适tf版本低，  
安装新环境conda create --name tf115 python=3.6  
找对应conda10.0版本conda install cudatoolkit=10.1
#### cuda版本查看  
cuda的版本不要nvcc --version查，查到的是之前安装的高版本，  
应在python中import tf之后print(tf.test.gpu_device_name())

#### 预训练模型                  
下载预训练模型，解压后模型位于'F:\zzqaq\data\chinese_wwm_ext_L-12_H-768_A-12'  

#### 终端调用
pip install bert-serving-server  
pip install bert-serving-client  这个要下。没下报错搞半天  
终端调用：F:\>bert-serving-start -model_dir   F:\zzqaq\data\chinese_wwm_ext_L-12_H-768_A-12 -num_worker=2  
bert-serving-start -model_dir   F:\zzqaq\data\FinBERT_L-12_H-768_A-12_tf\FinBERT_L-12_H-768_A-12_tf -num_worker=2  

jupyter需要改内核  

bert模型来源：https://github.com/ymcui/Chinese-BERT-wwm  
金融领域：https://github.com/valuesimplex/FinBERT

In [6]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [15]:
# 生成词向量，需终端调用
from bert_serving.client import BertClient
bc = BertClient()
doc_vecs = bc.encode(top_features_512.corpus.tolist())
doc_vecs.shape

(7781, 768)

In [3]:
# 原df
data_clean_2018_2020_path ='F:\\zzqaq\\data\\data_clean_2018_2020.csv'
data_clean_2018_2020 = pd.read_csv(data_clean_2018_2020_path,encoding="gb18030",converters={'code':str,'year':str,'board':str})
print(data_clean_2018_2020.shape)
data_clean_2018_2020[:2]

(7781, 4)


,code,year,text,board
0,000004,2018,讨论与分析一概述年度，公司实现主营业务收入万元，比上年同期增加，实现归属于上市公司股东的净利...,00
1,000004,2020,讨论与分析一概述年是公司业务转型后的第一年，年，公司筹划发行股份收购智游网安股权的重大资产重...,00


In [4]:
# 添加进df
corpus_index = data_clean_2018_2020.drop(columns=['text'])
corpus_new = pd.DataFrame(np.concatenate((corpus_index.values, doc_vecs_df), axis=1),
    columns=list(corpus_index.columns) + [f'feature_{i}' for i in range(768)])

In [22]:
# 保存
corpus_new.to_csv('F:\\zzqaq\\data\\merged_text_vecs_fin.csv',encoding="gb18030",index = False)

In [4]:
# 读取
text_vecs_path ='F:\\zzqaq\\data\\merged_text_vecs_fin.csv'
text_vecs = pd.read_csv(text_vecs_path,encoding="gb18030",converters={'code':str,'year':str,'board':str})
print(text_vecs.shape)
text_vecs.head(2)

(7781, 771)


,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_758,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767
0,000004,2018,00,-0.710000,0.082070,0.477866,0.057852,-0.391598,-0.628945,0.020472,...,0.712304,0.12474,-0.027108,-0.224301,0.113962,0.478764,-0.452287,0.260276,-0.280824,-0.278076
1,000004,2020,00,0.377358,-0.239015,0.122558,-0.083076,0.086279,-0.328752,-0.090525,...,0.678198,0.22263,0.201045,0.427176,-0.227349,0.221474,-0.725955,-0.090054,-1.034944,0.125335
